# Collecting Data of Huberman Podcast.

In [19]:
!pip install openai pytube -q
!pip install git+https://github.com/openai/whisper.git -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 1.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [20]:
import openai
import whisper
import pandas as pd
from pytube import YouTube
from getpass import getpass

COMPLETIONS_MODEL = "text-davinci-003"
EMBEDDINGS_MODEL = "text-embedding-ada-002"
# openai.api_key = getpass("")

In [21]:
model = whisper.load_model('base')

100%|████████████████████████████████████████| 139M/139M [00:01<00:00, 131MiB/s]


In [22]:
youtube_video_url = "https://www.youtube.com/watch?v=CQlTmOFM4Qs&t=996s"
youtube_video = YouTube(youtube_video_url)

In [23]:
dir(youtube_video)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_age_restricted',
 '_author',
 '_embed_html',
 '_fmt_streams',
 '_initial_data',
 '_js',
 '_js_url',
 '_metadata',
 '_player_config_args',
 '_publish_date',
 '_title',
 '_vid_info',
 '_watch_html',
 'age_restricted',
 'allow_oauth_cache',
 'author',
 'bypass_age_gate',
 'caption_tracks',
 'captions',
 'channel_id',
 'channel_url',
 'check_availability',
 'description',
 'embed_html',
 'embed_url',
 'fmt_streams',
 'from_id',
 'initial_data',
 'js',
 'js_url',
 'keywords',
 'length',
 'metadata',
 'publish_date',
 'rating',
 'register_on_complete_callback',
 'register_on_progress_callback',
 'stream_monostate',
 'streamin

In [24]:
stream = youtube_video.streams.filter(only_audio=True).first()
stream.download(filename='huberman_focus.mp4')

'/content/huberman_focus.mp4'

In [25]:
description = youtube_video.description

In [26]:
description

'In this episode, I discuss working memory, which is critical for learning and productivity, strategy setting, goal seeking, and navigating new environments. I explain the key role of dopamine and the biological mechanisms underlying working memory and how working memory differs from both short- and long-term memory.\n\nI also describe science-supported tools to enhance working memory and attention—including zero- or low-cost behavioral, supplemental and pharmacologic approaches. I include how to assess your working memory and how to use memory tasks to determine your baseline dopamine levels in certain brain circuits.\n\nThis episode provides listeners with highly actionable tools to assess and improve their attention to specific tasks and task-switching capacity and to enhance their overall productivity.\n\n*Thank you to our sponsors*\nAG1: https://drinkag1.com/huberman\nMateina: https://drinkmateina.com/huberman \nBetterHelp:  https://betterhelp.com/huberman \nHelix Sleep: https://h

In [28]:
transcribe = model.transcribe('/content/huberman_focus.mp4')
#  Took around 20 minutes for a single video. Will try using !nvidia-smi

/usr/local/lib/python3.10/dist-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


In [32]:
len(transcribe['text'])

99558

In [33]:
with open("/content/huberman_focus.txt", 'w') as file:
  file.write(transcribe['text'])

# Transcibing is taking long time therefore using datasets downloaded for LexFridman podcast on hugging face.

In [ ]:
# !pip install pyarrow --no-binary pyarrow
# !pip install datasets --no-build-isolation
# from datasets import load_dataset
# dataset = load_dataset("Whispering-GPT/lex-fridman-podcast")

In [3]:
# !pip install datasets
from datasets import load_dataset
dataset = load_dataset("Whispering-GPT/lex-fridman-podcast")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/346 [00:00<?, ? examples/s]

In [11]:
print(len(dataset["train"][0]['text']))


84308


In [ ]:
# DatasetDict({
#     train: Dataset({
#         features: ['id', 'channel', 'channel_id', 'title', 'categories', 'tags', 'description', 'text', 'segments'],
#         num_rows: 346
#     })
# })

In [44]:
len(dataset['train']['id'])

346

In [49]:
first_10_podcast=dataset["train"].select(range(10))
first_10_podcast

Dataset({
    features: ['id', 'channel', 'channel_id', 'title', 'categories', 'tags', 'description', 'text', 'segments'],
    num_rows: 10
})

In [51]:
# Import the required libraries; # !pip install reportlab; Can also use FPDF
# !pip install reportlab
from datasets import load_dataset
from reportlab.platypus import SimpleDocTemplate, Paragraph
from reportlab.lib.pagesizes import A4
from reportlab.lib.styles import getSampleStyleSheet

# Load the lex fridman dataset from hugging face
# dataset = load_dataset("Whispering-GPT/lex-fridman-podcast")


# Create a PDF document object
doc = SimpleDocTemplate("podcast.pdf", pagesize=A4, leftMargin=10, rightMargin=10, topMargin=20, bottomMargin=20)

# Get the sample style sheet
style = getSampleStyleSheet()

# Create an empty list to store the flowable objects
elements = []


# Loop through the first 10 episodes of the dataset
for episode in first_10_podcast:
    # Convert the string to a dictionary
    # Create a paragraph object for the title of the episode
    title = Paragraph(episode["title"], style["Heading1"])
    # Append the title to the elements list
    elements.append(title)
    # Create a paragraph object for the transcript of the episode
    text = Paragraph(episode["text"], style["Normal"])
    # Append the text to the elements list
    elements.append(text)


# Build the PDF document with the elements list
doc.build(elements)
